In [1]:
%matplotlib widget
import LinViscoFit as visco
GUI = visco.GUIControl()

from IPython.display import display, clear_output, HTML, Markdown

# Prony series identification for linear viscoelastic material models
Martin Springer | 2022 v0.1 

***
## Overview

Linear viscoelastic materials are often described with a Generalized Maxwell model. The necessary model parameter are identified by fitting a Prony series to the experimental measurement data. 

This Jupyter notebook allows for the identification of Prony series parameter from experimental data measured in either the frequency domain (via Dynamic Mechanical Thermal Analysis) or time domain (via relaxation measurements). The experimental data can be provided as raw measurement sets at different temperatures or as pre-processed master curves.

* If raw measurement data are provided, the time-temperature superposition principle is applied to create a master curve and obtain shift functions before the Prony series parameter are identified. 

* If master curves are provided, the shift procedure can be skipped and the Prony series parameters can be identified directly. 

An optional minimization routine is provided at the end of this notebook to reduce the number of Prony elements. This routine is helpful for Finite Element simulations where reducing the computational complexity of the linear viscoelastic material models can reduce the simulation time. 

In [2]:
import ipywidgets as widgets
from IPython.display import display
import markdown

with open('theory.md') as file:  
    md_theory = file.read() 
    
def show_theory(change):
    if change['new'] == True:
        out_HTML.value = markdown.markdown(md_theory)
    elif change ['new'] == False:
        out_HTML.value = ''

out_HTML = widgets.HTMLMath(
    value='',
)

b_theory = widgets.ToggleButton(
    value=False,
    description='Click here for more details!',
    layout = widgets.Layout(width = '200px'))


b_theory.observe(show_theory, 'value')

display(b_theory)
display(out_HTML)

ToggleButton(value=False, description='Click here for more details!', layout=Layout(width='200px'))

HTMLMath(value='')

***
## Parameter identification


### Specify measurement type and upload input data

In this section the measurement type is specified and the experimental data are uploaded. A set of example input files can be found here: [Example input files](https://github.com/martin-springer/LinViscoFit/tree/main/examples)

#### Conventions 
| Physical quantity        | Symbol               | Variable   | Unit   |
| :---------------------   | :-------------:      | :--------- | :----: |
| Relaxation modulus:      | $E(t)$               | `E_relax`  | MPa    |
| Storage modulus:         | $E'(\omega)$         | `E_stor`   | MPa    |
| Loss modulus:            | $E''(\omega)$        | `E_loss`   | MPa    |
| Complex modulus:         | $\lvert E^{*}\rvert$ | `E_comp`   | MPa    |
| Loss modulus:            | $\tan(\delta)$       | `tan_del`  | -      |
| Instantaneous modulus:   | $E_0$                | `E_0`      | MPa    |
| Equilibrium modulus:     | $E_{inf}$            | `E_inf`    | MPa    |
| Angular frequency:       | $\omega$             | `omega`    | rad/s  |
| Frequency:               | $f$                  | `f`        | 1/s    |
| Time:                    | $t$                  | `t`        | s      |
| Temperature:             | $\theta$             | `T`        | °C     |
| Relaxation times:        | $\tau  _i$           | `tau_i`    | s      |
| Relaxation moduli:       | $E_i$                | `E_i`      | MPa    |
| Norm. relaxation moduli: | $\alpha_i$           | `alpha_i`  | -      |

#### Domain
This notebook allows for the estimation of Prony series parameters used in a Generalized Maxwell model to describe  linear viscoelastic material behavior. The parameters can be either fitted from measurement data of Dynamic mechanical thermal analysis (DMTA) in the frequency domain (freq) or from relaxation experiments in the time domain (time). 

#### Instrument
DMTA measurements conducted with a Netzsch Gabo DMA EPLEXOR (Eplexor) can be directly uploaded as Excel files. Use the `Excel Export!` feature of the Eplexor software with the default template to create the input files. For measurements conducted with other instruments choose the user option (user) and preprocess the input files to be provided in comma-separated values (csv) format with the following columns header:
* **Frequency domain:** header = `f, E_stor, E_loss`, where `f` is the frequency in Hertz (Hz), and ```E_stor``` and `E_loss` are the storage and loss modulus in Megapascal (MPa), respectively.
* **Time domain:** header = `t, E_relax`, where `t` is the time in seconds (s), and `E_relax` is the relaxation modulus in MPa.

#### Type
The data can be provided as measurement sets at different temperatures (raw) or as master curve obtained from time-temperature superposition (master). If measurements at different temperatures are provided (raw) the individual temperature sets need to be identified. 
* **Eplexor** The notebook identifies the corresponding temperature sets automatically (only available in the frequency domain).
* **user** Two additional columns are necessary in the input file. One column indicating the temperature `T` of the measurement sets and one column with the header `Set` to identify the measurement sets. All measurements at the same temperature set are marked with the same number, e.g. 0 for all measurements in the the first temperature set. The first tempeature set (0) represents the coldest temperature followed by the second set (1) at the next higher temperature level and so forth (see the provided example input files for further details).

In [3]:
display(GUI.w_inp_gen)

#### Reference temperature 

Reference temperature refers to the temperature for which the master curve has been created through time-temperature superpostion. 

If a master curve from the Eplexor software is provided the reference temperature will be automatically extracted. 

For all other options please provide either the reference temperature of the user-specified master curve or the desired reference temperature for the time-temperature superposition of the raw data (for raw data the desired reference temperature will be automatically adjusted to align with the closest provided temperature set of the measurement input file).

#### Optional shift factor upload
If master curves are provided, the shift factor upload is optional and allows for the calculation of polynomial (D1 to D4) shift functions and the Williams–Landel–Ferry (WLF) shift function, but is not required for the Prony series estimation. 

If a master curve from created by the Eplexor software is provided, the default behavior of the notebook is to use the WLF shift function from the Eplexor software. However, in the time-temperature superpostion section, a checkbox is provided to overwrite the WLF fit of the Eplexor software and conduct another WLF fit with the algorithm in this notebook, which is then used to estimate the Prony series parameters.

If raw measurement data are provided (for both Eplexor or user), the shift factors can be either determined for the desired reference temperature in the time-temperature superposition section or user-specified shift factors can be uploaded to be used to create the master curve. 

In [4]:
display(GUI.w_inp_shift)

#### Load and check the provided input files

Once all the input parameter are specified and the necessary files are uploaded click the *Load data* button below. If the input files could be loaded successfully, the widgets will indicate the state of the uploaded data.

In [5]:
display(GUI.w_inp_load)
#display(GUI.w_inp_check)

***
### Time-temperature superposition (shift functions)

This section allows to calulate shift factors from raw input data and fit shift functions.

#### Shift factors $\log(a_{T})$ - Create master curve from raw input

This subsection only applies if raw input data have been provided. If master curves have been uploaded, please procede to the next step. 

Here, the time-temperature superposition principle is applied to create a master curve from the individual temperature measurement sets provided in the raw input data file. If user shift factors, $\log(a_{T})$, are uploaded, the provided shift factors will be used to create the master curve from the raw measurement sets at different temperatures (the checkbox allows you to overwrite the provided shift factors). If no shift factors are provided, the measurement sets from the raw input file are used to estimate the shift factors and create a master curve. Measurement sets below the desired reference temperatures are shifted to lower frequencies (longer time periods), whereas measurement sets at temperatures higher than the reference temperature are shifted to higher frequencies (shorter time periods). For the frequency domain, only the storage modulus input data are considered to create the master curve from the raw input data. The shift factors obtained from creating the storage modulus master curve are then used to create the loss modulus master curve.

In [6]:
display(GUI.w_aT)

#### Shift functions (WLF & Polynomial degree 1 to 4)

If shift factors are available, the WLF shift function and polynomial functions of degree 1 to 4 can be fitted and plotted below. If the WLF shift functions was already provided by the Eplexor software, the checkbox below let's you overwrite the WLF fit of the Eplexor software with a WLF fit of this notebook. Either way, both solutions should be very similar. 

In [7]:
display(GUI.w_shift)

***
### Estimate Prony series parameters

#### Pre-process (smooth) master curve

A moving median filter to remove outliers in the measurement data can be applied before the Prony series parameters are identified. The window size (win) can be adjusted through the slider above the figure. A window size of 1 means that no filtering procedure is performed and the raw data are fitted.

In [8]:
display(GUI.b_smooth)
display(GUI.out_smooth)

Button(button_style='info', description='smooth master curve', layout=Layout(height='auto', width='200px'), st…

Output()

#### Define the number and discretization of the Prony series

The number of Prony terms, $N$, needs to be defined before the parameter $\tau_i$ and $\alpha_i$ can be identified. The default behavior is to equally space one Prony term per decade along the logarithmic time axis, $\tau_i$ = [1E-1, 1E0, 1E1,...] (s), within the experimental window. This discretization typically delivers accurate results for engineering applications. However, the fine discretization can be computationally heavy for usage in Finite Element simulations. Hence, we allow the user to modify this default discretization by either using the optimization routine provided below or by manually defining the number of Prony terms $N$ for the discretization. Additionally, the user can decide wether to round the lowest and highest relaxation times, $\tau_i$, to the nearest base 10 number within the measurement window (round) or to use the exact minimum and maximum values of the experimental window for the relaxation times (exact). 

In [9]:
display(GUI.w_dis)

#### Curve fitting

Two different curve fitting routines for the Prony series parameters are employed and are dependent on the domain of the input data:

* **Frequency domain**: A generalized collocation method using stiffness matrices is used as described in [Kraus, M. A., and M. Niederwald. Eur J Eng Mech 37.1 (2017): 82-106](https://journals.ub.ovgu.de/index.php/techmech/article/view/600). This methods utilizes both the storage and loss modulus master curves to estimate the Prony series parameters.
   
   
* **Time domain**: A least-squares minimization is performed with the L-BFGS-B method from the scipy package. The implementation is similar to the optimization problem described by [Barrientos, E., Pelayo, F., Noriega, Á. et al. Mech Time-Depend Mater 23, 193–206 (2019)](https://doi.org/10.1007/s11043-018-9394-z) for a homogenous distribution of discrete times. 

In [10]:
display(GUI.b_fit)
#display(GUI.out_fit)
display(GUI.w_out_fit_prony)

Button(button_style='danger', description='fit Prony series', layout=Layout(height='auto', width='200px'), sty…

***
#### Generalized Maxwell model

The fitted Prony series parameters in combination with the Generalized Maxwell model can be used to calculate the linear viscoelastic material parameters in both the time and frequency domain. 

In [11]:
display(GUI.b_GMaxw)
display(GUI.out_GMaxw)

Button(button_style='info', description='plot Generalized Maxwell', layout=Layout(height='auto', width='200px'…

Output()

***
### Optional: Minimize number of Prony elements (for Finite Element simulations)

The Generalized Maxwell model in combination with a high number of Prony terms can be computationally expensive. Especially, when used in combination with numerical frameworks as the Finite Element Method. Reducing the number of Prony elements will decrease the accurace of the linear viscoelastic material model, but can help to speed up subsequent numerical simulations.

We provide a simple routine to create an additional Generalized Maxwell model with a reduced number of Prony elements. The routine starts with the number of Prony terms specified above and subsequently reduces the number of terms. A least squares minimization is performed to fit the reduced term Prony parameters. The least squares residual is used to suggest an optimal number of Prony terms for subsequent FEM simulations ($R^2 \approx$ 0.025). However, the user can change this default setting by selecting a different number of Prony terms below. The selected model will be exported in addition to the model fitted above.

> **_Note:_** This routine takes longer to complete and can run for a couple of minutes. The runtime depends on the initial number of Prony elements and the number of data points in the measurement sets.

In [12]:
display(GUI.b_opt)
display(GUI.w_out_fit_min)
#display(GUI.out_res)
#display(GUI.out_opt)

Button(button_style='info', description='minimize Prony terms', layout=Layout(height='auto', width='200px'), s…

***
### Download

The fitted Prony terms and dataframes used for the calculations can be downloaded as zip archive. All figures generated in this notebook are included as well.

In [13]:
display(GUI.db_zip)

Button(button_style='warning', description='Download zip', layout=Layout(height='auto', width='200px'), style=…

In [14]:
#old
#display(GUI.db_prony)

***
### Start over

Clear all input data and load an empty notebook.

In [15]:
display(GUI.b_reload)
display(GUI.out_html)

Button(button_style='danger', description='Clear notebook!', layout=Layout(height='auto', width='200px'), styl…

Output()

***
## Verification - Compare with Ansys